In [2]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

/home/max/micromamba/envs/pol_stats/lib/python3.12/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "cannot import name 'Callable' from 'collections' (/home/max/micromamba/envs/pol_stats/lib/python3.12/collections/__init__.py)".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

$\dot{A}=gA$ 

$\dot{g}=-\alpha(g-\beta \frac{A_{end}-A}{A_{end}})$ if $A<A_{cut}$

Combine Dev and Reg with differnt initial conditions

In [3]:
rng = np.random.default_rng()

from scipy.integrate import solve_ivp

def A_theor(t, A_0, g_0, alpha, beta, A_end,A_cut):
    t48 = t - 48

    def ode_system(t, y):
        A, g = y
        dAdt = g*A
        if A < A_cut:
            dgdt = -alpha * (g - beta * (A_end - A_cut) / A_end)
        else:
            dgdt = -alpha * (g - beta * (A_end - A) / A_end)
        return [dAdt, dgdt]

    # Initial conditions
    y0 = [A_0, g_0]

    # Solve the ODE
    solution = solve_ivp(ode_system, [t48.min(), t48.max()], y0, t_eval=t48, method='RK45')
    #print("Solver message:", solution.message)
    return solution.y[0]

def linCrit_ppc(x):
    sigma = 0*np.abs(rng.normal(0, 2.0))
    A_0 = 10**rng.normal(0.3, 0.15)
    g_0 = np.abs(rng.normal(0, 0.1))
    alpha = 10**rng.normal(-0.25, 0.25)
    beta = (alpha/4)*10**rng.normal(0, 0.1)
    A_end = 10**rng.normal(1.0, 0.1)
    A_cut = 2 + 4*rng.beta(5.0,5.0)
    #print(A_0, g_0,alpha, beta, A_end)
    return (rng.normal(A_theor(x, A_0, g_0,alpha, beta, A_end,A_cut), sigma),
            {"A_0": A_0, "g_0": g_0, "alpha": alpha, "beta": beta, "A_end": A_end, "A_cut":A_cut})

In [4]:
x = np.linspace(48, 144, 30)
for _ in range(10):
    print(linCrit_ppc(x))

(array([2.09125535, 2.89083996, 3.5461324 , 4.28592148, 5.17050145,
       6.18005931, 7.15034957, 7.97927946, 8.62716324, 9.09796001,
       9.42103372, 9.63303215, 9.76770816, 9.85153309, 9.90300008,
       9.93425285, 9.95313253, 9.96451376, 9.97131772, 9.97543912,
       9.97786891, 9.9793548 , 9.98023839, 9.98076513, 9.98107759,
       9.98127157, 9.98137473, 9.98145136, 9.98149189, 9.98151412]), {'A_0': 2.0912553456005645, 'g_0': np.float64(0.15470369967849049), 'alpha': 0.6300048596626002, 'beta': 0.11689603406365542, 'A_end': 9.981548372016583, 'A_cut': 5.148960459126133})
(array([2.16026738, 4.78390286, 8.60286312, 9.74631559, 9.83947613,
       9.84054308, 9.84022309, 9.84020014, 9.84020117, 9.84020489,
       9.84019347, 9.84020338, 9.84020073, 9.84019498, 9.84020138,
       9.84020103, 9.8401998 , 9.84018104, 9.84022465, 9.84020077,
       9.84019889, 9.84019422, 9.84020671, 9.84020473, 9.84019984,
       9.84019886, 9.84020593, 9.8402017 , 9.84019879, 9.84020059]), {'A_0':

In [5]:
x = np.linspace(48, 144, 100)
f_ppc = [linCrit_ppc(x) for _ in range(100)]

p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

for f, params in f_ppc:
    line = p.line(x, f)
    p.add_tools(bokeh.models.HoverTool(
        renderers=[line],
        tooltips=[(key, f"{params[key]}") for key in params.keys()]
    ))

bokeh.io.show(p)

In [37]:
import os
Sahra_Shivani_path=(r'/media/max_kotz/sahra_shivani_data')
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
hdf5_file_path = os.path.join(Curv_Thick_path, 'scalarGrowthData.h5')

# Aoad the DataFrame from the HDF5 file
df = pd.read_hdf(hdf5_file_path, key='data')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="Surface Area",
    #x_axis_type="log",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_5
items = []
print(df)
for color, (genotype, sub_df) in zip(colors, df.groupby("condition")):
    print(genotype)
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["Surface Area"]*1e-4,
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)


                             Mask Folder        Volume   Surface Area  \
0    20240409sox10_clau132hpfdev3_Stitch  2.156393e+06  121968.863980   
1    20240410sox10_clau144hpfdev1_Stitch  1.949042e+06  112923.567433   
2    20240103sox10_clau126hpfreg4_Stitch  2.121201e+06  102161.069891   
3    20240103sox10_clau126hpfreg3_Stitch  2.013371e+06   87418.650875   
4    20240329sox10_clau144hpfdev1_Stitch  1.920304e+06  103024.294823   
..                                   ...           ...            ...   
185     20240626sox10_clau120hpfreg1ff72  2.098723e+06   98366.658752   
186     20240626sox10_clau120hpfreg2ff72  2.280549e+06   91374.312464   
187     20240626sox10_clau120hpfreg3ff72  2.406311e+06  105994.062418   
188     20240626sox10_clau120hpfreg4ff72  2.390018e+06   94558.304416   
189     20240626sox10_clau120hpfreg6ff72  2.419007e+06  107340.797662   

     Integrated Thickness        L PD        L AP     condition  time in hpf  \
0            2.102532e+06  479.895636  339.

In [38]:
sm = cmdstanpy.CmdStanModel(stan_file='fit_setPoint.stan')

In [39]:
def df_to_data(cat_Dev,cat_Reg):
    filtered_df = df[df['condition'] == cat_Reg]
    t_Reg = filtered_df['time in hpf'].to_numpy()
    A_Reg = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Reg)
    t_Reg = t_Reg[sorted_indices]
    A_Reg = A_Reg[sorted_indices]

    filtered_df = df[df['condition'] == cat_Dev]
    t_Dev = filtered_df['time in hpf'].to_numpy()
    A_Dev = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Dev)
    t_Dev = t_Dev[sorted_indices]
    A_Dev = A_Dev[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(48, 144, N_ppc)
    
    return dict(t_Reg=t_Reg, A_Reg=A_Reg, N_Reg=len(t_Reg),
                t_Dev=t_Dev, A_Dev=A_Dev, N_Dev=len(t_Dev),
                  N_ppc=N_ppc, t_ppc=t_ppc)

data = df_to_data('Development','Regeneration')

In [40]:
samples = sm.sample(data=data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['A_Dev_ppc','A_Reg_ppc'])
bebi103.stan.check_all_diagnostics(samples)

13:19:02 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

13:19:41 - cmdstanpy - INFO - CmdStan done processing.
13:19:41 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: ode_rk45:  Failed to integrate to next output time (48) in less than max_num_steps steps (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45:  Failed to integrate to next output time (48) in less than max_num_steps steps (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45:  Failed to integrate to next output time (60) in less than max_num_steps steps (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45: initial state[1] is inf, but must be finite! (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Excep


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

0 of 4000 (0.0%) iterations ended with a divergence.

0 of 4000 (0.0%) iterations saturated the maximum tree depth of 10.

E-BFMI indicated no pathological behavior.


0

In [41]:
output_file("area_DevREG_ppc_corner_Dev.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end','A_cut','A_0_Dev','g_0_Dev', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [42]:
output_file("area_DevREG_ppc_corner_Reg.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end','A_cut','A_0_Reg','g_0_Reg', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [ ]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [43]:
output_file("area_DevREG_ppc_At_Dev.html")
f_ppc = samples.posterior_predictive.A_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Dev"], data["A_Dev"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [44]:
output_file("area_DevREG_ppc_At_Reg.html")
f_ppc = samples.posterior_predictive.A_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Reg"], data["A_Reg"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [45]:
posterior = samples.posterior

# Dictionary to store mean and confidence intervals for each variable
results ={}

# Loop through each variable in the posterior
for var_name in posterior.data_vars:
    data = posterior[var_name].values.flatten()
    
    results[var_name]=data

print(results)
# Convert the results dictionary to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the table
print(results_df)
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
results_df.to_csv(os.path.join(Curv_Thick_path,"area_sampled_parameter_results_setPoint.csv"), index=False)

{'A_0_Dev': array([1.7865 , 2.17307, 2.12803, ..., 2.5094 , 2.35498, 1.82644]), 'A_0_Dev_tilde': array([0.252002, 0.337074, 0.327978, ..., 0.399569, 0.371988, 0.261605]), 'A_0_Reg': array([1.46849, 1.27421, 1.27494, ..., 1.59765, 1.53051, 1.50248]), 'A_0_Reg_tilde': array([0.166872, 0.105242, 0.105489, ..., 0.203481, 0.184836, 0.176808]), 'A_cut': array([4.70218, 4.56647, 4.3901 , ..., 4.53789, 4.41823, 4.10953]), 'A_cut_tilde': array([0.73337 , 0.582395, 0.395166, ..., 0.55145 , 0.424488, 0.109637]), 'A_end': array([10.7689, 10.9937, 11.0903, ..., 10.8203, 10.8325, 11.0963]), 'A_end_tilde': array([1.03217, 1.04114, 1.04494, ..., 1.03424, 1.03473, 1.04518]), 'alpha': array([0.357696, 0.369542, 0.367448, ..., 0.318509, 0.331996, 0.308281]), 'alpha_tilde': array([-0.446486, -0.432337, -0.434804, ..., -0.496878, -0.478867,
       -0.511053]), 'beta_': array([0.0678449, 0.0669066, 0.0727133, ..., 0.0679027, 0.0739656,
       0.0689642]), 'beta_tilde': array([-0.119937 , -0.140134 , -0.1015

In [ ]:
bebi103.stan.clean_cmdstan()